In [1]:
import polars as pl
from mimesis import Person, Address, Numeric
import xlsxwriter
import duckdb

In [2]:
peep = Person()
adrs = Address()
num = Numeric()

In [3]:
data = []
for _ in range(100_000):
    data.append({'name':peep.full_name(), 'address':adrs.address(), 'zip_cd':adrs.zip_code()
                 , 'state':adrs.state(), 'net_worth':num.integer_number(5000,50000)})

In [4]:
df = pl.DataFrame(data)

In [5]:
df.head(5)

name,address,zip_cd,state,net_worth
str,str,str,str,i64
"""Lynna Vang""","""1054 Miller Ce…","""84769""","""Tennessee""",26274
"""Moriah Harmon""","""1318 Jeopardy …","""68278""","""Pennsylvania""",36192
"""Kortney Burns""","""999 Saint Loui…","""99648""","""Ohio""",47950
"""Minna Fulton""","""477 Marvel Est…","""65209""","""Texas""",16183
"""Bobette Gallag…","""86 Dunsmuir Ru…","""31921""","""North Carolina…",12086


In [57]:
res = df.group_by(pl.col('zip_cd')).agg(pl.col('name').n_unique().name.suffix("_cnt")
                                        ,pl.col('address').count().name.suffix("_cnt")
                                        ,pl.col('net_worth').sum().name.suffix("_tot")
                            )
print(res)

shape: (63_168, 4)
┌────────┬──────────┬─────────────┬───────────────┐
│ zip_cd ┆ name_cnt ┆ address_cnt ┆ net_worth_tot │
│ ---    ┆ ---      ┆ ---         ┆ ---           │
│ str    ┆ u32      ┆ u32         ┆ i64           │
╞════════╪══════════╪═════════════╪═══════════════╡
│ 95887  ┆ 4        ┆ 4           ┆ 79981         │
│ 78005  ┆ 1        ┆ 1           ┆ 13615         │
│ 88794  ┆ 1        ┆ 1           ┆ 49062         │
│ 40225  ┆ 1        ┆ 1           ┆ 31707         │
│ 06088  ┆ 1        ┆ 1           ┆ 5785          │
│ …      ┆ …        ┆ …           ┆ …             │
│ 90521  ┆ 2        ┆ 2           ┆ 74368         │
│ 46583  ┆ 1        ┆ 1           ┆ 47269         │
│ 19442  ┆ 1        ┆ 1           ┆ 48631         │
│ 32625  ┆ 2        ┆ 2           ┆ 72147         │
│ 71798  ┆ 2        ┆ 2           ┆ 51017         │
└────────┴──────────┴─────────────┴───────────────┘


In [62]:
res.write_parquet("./agg_pl.parquet")
res.write_excel("./test.xlsx")

In [8]:
sql = """
    select zip_cd, count(distinct address) as adrs_cnt
        ,count(distinct name) as nm_cnt
        ,sum(net_worth) as net_worth_tot
    from df
    group by 1
"""

duckdb.sql(sql).show()

┌─────────┬──────────┬────────┬───────────────┐
│ zip_cd  │ adrs_cnt │ nm_cnt │ net_worth_tot │
│ varchar │  int64   │ int64  │    int128     │
├─────────┼──────────┼────────┼───────────────┤
│ 30464   │        1 │      1 │         49028 │
│ 70790   │        2 │      2 │         93377 │
│ 61547   │        3 │      3 │         63119 │
│ 77274   │        1 │      1 │         14577 │
│ 41876   │        2 │      2 │         49842 │
│ 87568   │        1 │      1 │          7825 │
│ 35258   │        2 │      2 │         72500 │
│ 47701   │        1 │      1 │         23175 │
│ 31944   │        2 │      2 │         38813 │
│ 38623   │        1 │      1 │          8832 │
│   ·     │        · │      · │            ·  │
│   ·     │        · │      · │            ·  │
│   ·     │        · │      · │            ·  │
│ 46924   │        1 │      1 │         31305 │
│ 69980   │        1 │      1 │          7908 │
│ 56105   │        1 │      1 │         47937 │
│ 52511   │        1 │      1 │         